In [ ]:
!pip install openai
!pip install sentence-transformers
!pip install langchain pypdf langchain-openai #tiktoken chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuso

# Embeddings

In [ ]:
import pandas as pd

df = pd.read_csv('http://bit.ly/dataset-sst2',
                 nrows=100, sep='\t', names=['text', 'label'])

df['label'] = df['label'].replace({0: 'negative', 1: 'positive'})

In [ ]:
df.head()

,text,label
0,"a stirring , funny and finally transporting re...",positive
1,apparently reassembled from the cutting room f...,negative
2,they presume their audience wo n't sit still f...,negative
3,this is a visually stunning rumination on love...,positive
4,jonathan parker 's bartleby should have been t...,positive


In [ ]:
from sentence_transformers import SentenceTransformer

sentence_bert_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

def get_embeddings(sentences):
    return sentence_bert_model.encode(sentences,
                                    batch_size=32,
                                    show_progress_bar=True)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.05k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
e = get_embeddings(df['text'])

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Convert NumPy array of embedding into data frame
embedding_df = pd.DataFrame(e)

# Save dataframe as as TSV file without any index and header
embedding_df.to_csv('output.tsv', sep='\t', index=None, header=None)

In [ ]:
embedding_df.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.007687,0.220588,1.107231,-0.805426,-0.189138,0.175037,-0.095774,-1.149741,0.614675,-0.406313,...,-0.091129,-0.930856,-0.770217,0.830264,-0.214198,0.400544,0.974434,-0.599894,0.419342,-0.242024
1,0.131850,-1.051063,0.379231,-0.066255,-0.299081,-0.981429,0.958279,-0.036092,-0.081783,-0.722954,...,0.048544,-0.194993,0.429363,1.433621,0.303520,-0.217460,-0.337119,-0.513364,0.613231,0.939613
2,0.146425,0.026571,0.910159,-0.720145,-0.057320,0.184522,-0.354749,-0.819164,0.438382,-0.083151,...,-0.953810,0.045402,-0.156494,-0.422208,-0.472993,0.141547,0.079621,-0.068062,-0.347549,-0.200186
3,0.725216,0.400027,0.550634,-0.415072,0.321566,0.476579,-0.268125,0.285532,-0.096131,-0.754695,...,-0.834957,-0.458781,-0.016415,0.061813,-0.027487,0.248661,0.333310,-0.202888,-0.089137,-0.841737
4,0.386161,0.437596,-0.229873,-0.328632,0.146723,0.166341,0.614475,-0.451549,-0.415271,0.054353,...,-0.144842,-0.170682,0.482448,0.865260,0.456433,-0.029702,0.791408,0.490984,0.352877,0.135437


In [ ]:
# Save dataframe without any index
df.to_csv('metadata.tsv', index=False, sep='\t')

Embeddings projector link: https://projector.tensorflow.org/

# Standard imports

In [ ]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)


# RAG

In [ ]:
!pip install llama-index --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install pypdf

In [ ]:
!wget https://www.goldmansachs.com/intelligence/pages/gs-research/2024-us-equity-outlook-all-you-had-to-do-was-stay/report.pdf

--2024-07-26 00:33:13--  https://www.goldmansachs.com/intelligence/pages/gs-research/2024-us-equity-outlook-all-you-had-to-do-was-stay/report.pdf
Resolving www.goldmansachs.com (www.goldmansachs.com)... 23.204.24.145
Connecting to www.goldmansachs.com (www.goldmansachs.com)|23.204.24.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491250 (480K) [application/pdf]
Saving to: ‘report.pdf’

report.pdf          100%[===================>] 479.74K  --.-KB/s    in 0.04s   

2024-07-26 00:33:13 (12.8 MB/s) - ‘report.pdf’ saved [491250/491250]



In [ ]:
import os
os.environ["OPENAI_API_KEY"] = open_ai_key

In [ ]:
# Import necessary classes from the llama_index package
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Read documents from the specified directory and load a specific document, "report.pdf".
documents = SimpleDirectoryReader("./").load_data("report.pdf")

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
index = VectorStoreIndex.from_documents(documents)

# Convert the VectorStoreIndex object into a query engine. This query engine can be used to
# perform semantic searches on the index, matching natural language queries to the most relevant
# documents in the index.
query_engine = index.as_query_engine()

# Use the query engine to search for documents that are relevant to the query
# from the indexed documents based on the semantic understanding of the query.
response = query_engine.query("What is the 2024 outlook for US GDP?")

# Print the response obtained from the query. This will display the result of the semantic search,
# showing the information or documents that best match the query about the 2024 outlook.
print(response)


Loading files: 100%|██████████| 3/3 [00:02<00:00,  1.19file/s]


The 2024 outlook for US GDP is forecasted to have above-consensus full-year GDP growth of 2.1%.


# Adding memory

In [ ]:
chat_engine = index.as_chat_engine(chat_mode="openai", verbose=True)
response = chat_engine.chat("Hi there")
print(response)

Added user message to memory: Hi there
Hello! How can I assist you today?


In [ ]:
response = chat_engine.chat("What is the liklihood of recession in 2024?")
print(response)

Added user message to memory: What is the liklihood of recession in 2024?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"What is the likelihood of a recession in 2024?"}
Got output: The likelihood of a recession beginning in 2024, according to the Goldman Sachs economics view, is 15%.

The likelihood of a recession beginning in 2024, according to the Goldman Sachs economics view, is 15%.


In [ ]:
response = chat_engine.chat("Can you expand on that")
print(response)

Added user message to memory: Can you expand on that
Goldman Sachs economists have estimated a 15% chance of a recession occurring in 2024. This prediction is based on various economic indicators, trends, and forecasts that they analyze to assess the potential risks and factors that could lead to a recession in that year. It's important to note that economic predictions are subject to change based on evolving conditions and data.


In [ ]:
chat_engine.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='Hi there', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Hello! How can I assist you today?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the liklihood of recession in 2024?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, additional_kwargs={'tool_calls': [ChatCompletionMessageToolCall(id='call_DL5QIChZ2bBkkuH1p3NDqFDM', function=Function(arguments='{"input":"What is the likelihood of a recession in 2024?"}', name='query_engine_tool'), type='function')]}),
 ChatMessage(role=<MessageRole.TOOL: 'tool'>, content='The likelihood of a recession beginning in 2024, according to the Goldman Sachs economics view, is 15%.', additional_kwargs={'name': 'query_engine_tool', 'tool_call_id': 'call_DL5QIChZ2bBkkuH1p3NDqFDM'}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='The likelihood of a recession

# Chunking

In [ ]:
from llama_index.core import ServiceContext

service_context = ServiceContext.from_defaults(chunk_size=1000)

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./").load_data("report.pdf")
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)
query_engine = index.as_query_engine()
response = query_engine.query("What is the likelihood of a recession in 2024?")
print(response)

# Vector DB

In [ ]:
%pip install llama-index-vector-stores-weaviate

In [ ]:
!pip install llama-index-vector-stores-chroma

In [ ]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.create_collection("tech16example")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./").load_data("report.pdf")
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
query_engine = index.as_query_engine()
response = query_engine.query("What is the 2024 outlook for US GDP?")
print(response)